In [1]:
!pip install transformers==4.30
!pip install datasets
!pip install einops
!pip install json
!pip install tqdm
!pip install fire
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install wandb
!pip install jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import wandb
wandb.login()
#d772f5f38ab10f614d332405f1228597362da736

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# import os
# import sys

# import numpy
# import pandas as pd
# import torch
# import transformers

# import json

# from transformers import AutoModelForCausalLM, AutoTokenizer
# from datasets import load_dataset

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# torch.set_default_device("cuda")
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
# inputs = tokenizer('''```python
#     def print_prime(n):
#     """
#     Print all primes between 1 and n
#     """''', return_tensors="pt", return_attention_mask=False)

# outputs = model.generate(**inputs, max_length=200)
# text = tokenizer.batch_decode(outputs)[0]
# print(text)

In [3]:
from datasets import load_dataset
import json
from transformers import AutoTokenizer
from tqdm import tqdm

print('시작')
data = load_dataset('codeparrot/apps')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

train = []
test = []

print('train 시작')
for data_point in tqdm(data['train']):
    try:
        question = data_point['question']
        if len(tokenizer(question).input_ids) > 512:
            continue
        inputs = json.loads(data_point['input_output'])['inputs']
        output = json.loads(data_point['input_output'])['outputs']
        cnt = 0
        for solution in eval(data_point['solutions']):
            if cnt == 1:
                break
            if len(tokenizer(solution).input_ids) > 512:
                continue
            cnt +=1
            train.append({'description' : question, 'solution' : solution, 'input' : inputs, 'output' : output})
    except:
        continue

with open("./apps_train.jsonl" , encoding= "utf-8",mode="w") as file:
    for line in train[:1500]:
        file.write(json.dumps(line))
        file.write('\n')

print('test 시작')

for data_point in tqdm(data['test']):
    try:
        question = data_point['question']
        if len(tokenizer(question).input_ids) > 512:
            continue
        inputs = json.loads(data_point['input_output'])['inputs']
        output = json.loads(data_point['input_output'])['outputs']
        test.append({'description' : question, 'solution' : '', 'input' : inputs, 'output' : output})
    except:
        continue
print(cnt)
with open("./apps_test.jsonl" , encoding= "utf-8",mode="w") as file:
    for line in test:
        file.write(json.dumps(line))
        file.write('\n')

print('완료')

시작


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

train 시작


100%|██████████| 5000/5000 [00:09<00:00, 512.77it/s]


test 시작


100%|██████████| 5000/5000 [00:17<00:00, 288.06it/s]


1
완료


In [4]:
import torch
from torch.utils.data import Dataset
import json
from datasets import load_dataset

class CodeDataset(Dataset):
    def __init__(self, file_path):
        self.examples = []
        self.data = []

        with open(file_path, "r") as f:
            for line in f:
                example = json.loads(line)
                self.examples.append(example)
        for example in self.examples:
            self.data.append(
                {
                    "input" : example['description'],
                    "output" : example["solution"],
                }
            )

    def map(self, func):
        self.data = [func(item) for item in self.data]


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        return self.data[idx]

In [5]:
import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: Union[None, str] = None,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if not instruction and input:
            res = self.template["prompt_no_instruction"].format(input=input)
        elif input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        # if input:
        #     res = self.template["prompt_input"].format(
        #         input=input
        #     )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1]

    def get_input(self, output: str) -> str:
        return output.split(self.template["response_split"])[0]

In [7]:
import os
import sys
from typing import List
import jsonlines
import bitsandbytes as bnb

import numpy as np
import fire
import torch
import transformers
from datasets import load_dataset

"""
Unused imports:
import torch.nn as nn
import bitsandbytes as bnb
"""

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainerCallback, get_cosine_schedule_with_warmup


# class LogCallback(TrainerCallback):
#     def init(self, state):
#         self.state = state
#     def on_step_end(self, args, state, control, logs = None, logging_steps=1, **kwargs):
#         if state.global_step % logging_steps == 0:
#             # 매 logging_steps 스텝마다 training loss 출력
#             print(f"Step {state.global_step}: Train loss {state.log_history[-1]}")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def train(
    # model/data params
    base_model: str = "microsoft/phi-1_5",  # the only required argument
    data_path: str = "./",
    output_dir: str = "/content/drive/MyDrive/out/first/",
    # training hyperparams
    batch_size: int = 128,
    micro_batch_size: int = 1,
    num_epochs: int = 3,
    learning_rate: float = 3e-4,
    cutoff_len: int = 1024,
    val_set_size: int = 2000,
    # lora hyperparams
    # lora_r: int = 16,
    lora_r: int =32,
    lora_alpha: int = 16,
    lora_dropout: float = 0.05,
    # lora_target_modules: List[str] = [
    #     "q_proj",
    #     "v_proj",
    # ],
    lora_target_modules: List[str] = ['Wqkv', 'out_proj'],
    # llm hyperparams
    train_on_inputs: bool = False,  # if False, masks out inputs in loss
    add_eos_token: bool = True,
    group_by_length: bool = False,  # faster, but produces an odd training loss curve
    # wandb params
    wandb_project: str = "dlproject",
    wandb_run_name: str = "first",
    wandb_watch: str = "true",  # options: false | gradients | all
    wandb_log_model: str = "true",  # options: false | true
    resume_from_checkpoint: str = None,  # either training checkpoint or final adapter
    prompt_template_name: str = "no_instruction",  # The prompt template to use, will default to alpaca.
    seed: int = 42,
):
    if int(os.environ.get("LOCAL_RANK", 0)) == 0:
        print(
            f"Training Alpaca-LoRA model with params:\n"
            f"base_model: {base_model}\n"
            f"data_path: {data_path}\n"
            f"output_dir: {output_dir}\n"
            f"batch_size: {batch_size}\n"
            f"micro_batch_size: {micro_batch_size}\n"
            f"num_epochs: {num_epochs}\n"
            f"learning_rate: {learning_rate}\n"
            f"cutoff_len: {cutoff_len}\n"
            f"val_set_size: {val_set_size}\n"
            f"lora_r: {lora_r}\n"
            f"lora_alpha: {lora_alpha}\n"
            f"lora_dropout: {lora_dropout}\n"
            f"lora_target_modules: {lora_target_modules}\n"
            f"train_on_inputs: {train_on_inputs}\n"
            f"add_eos_token: {add_eos_token}\n"
            f"group_by_length: {group_by_length}\n"
            f"wandb_project: {wandb_project}\n"
            f"wandb_run_name: {wandb_run_name}\n"
            f"wandb_watch: {wandb_watch}\n"
            f"wandb_log_model: {wandb_log_model}\n"
            f"resume_from_checkpoint: {resume_from_checkpoint or False}\n"
            f"prompt template: {prompt_template_name}\n"
        )
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"
    gradient_accumulation_steps = batch_size // micro_batch_size
    # seed 설정
    set_seed(seed)

    prompter = Prompter(prompt_template_name)

    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        gradient_accumulation_steps = gradient_accumulation_steps // world_size

    # Check if parameter passed or if set within environ
    use_wandb = len(wandb_project) > 0 or (
        "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
    )
    # Only overwrite environ if wandb param passed
    if len(wandb_project) > 0:
        os.environ["WANDB_PROJECT"] = wandb_project
    if len(wandb_watch) > 0:
        os.environ["WANDB_WATCH"] = wandb_watch
    if len(wandb_log_model) > 0:
        os.environ["WANDB_LOG_MODEL"] = wandb_log_model

    # test할 때는 wandb 사용하지 않음
    # use_wandb = None
    wandb.init(project='dlproject')
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    global model
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        device_map={"":0},
        trust_remote_code=True,
        quantization_config=bnb_config
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.padding_side = "left"  # Allow batched inference
    tokenizer.pad_token_id = tokenizer.eos_token_id

    def tokenize(prompt, add_eos_token=True):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

    def generate_and_tokenize_prompt(data_point):
        full_prompt = prompter.generate_prompt(
            input = data_point["input"],
            label = data_point["output"],
        )
        tokenized_full_prompt = tokenize(full_prompt, add_eos_token=add_eos_token)
        if not train_on_inputs:
            user_prompt = prompter.generate_prompt(
                input = data_point["input"]
            )
            tokenized_user_prompt = tokenize(
                user_prompt, add_eos_token=add_eos_token
            )
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        # inputs = data_point["input"]
        # targets = data_point["output"]
        # model_inputs = tokenizer(inputs, max_length=cutoff_len, padding="max_length", truncation=True, return_tensors="pt")
        # labels = tokenizer(targets, max_length=cutoff_len, padding="max_length", truncation=True, return_tensors="pt")
        # labels = labels["input_ids"]
        # labels[labels == tokenizer.pad_token_id] = -100
        # model_inputs["labels"] = labels
        return tokenized_full_prompt

    def generate_and_tokenize_prompt_for_test(data_point):
        full_prompt = prompter.generate_prompt(
            input = data_point["input"],
        )
        tokenized_full_prompt = tokenize(full_prompt, add_eos_token=add_eos_token)
        if not train_on_inputs:
            user_prompt = prompter.generate_prompt(
                input = data_point["input"]
            )
            tokenized_user_prompt = tokenize(
                user_prompt, add_eos_token=add_eos_token
            )
            user_prompt_len = len(tokenized_user_prompt["input_ids"])

            if add_eos_token:
                user_prompt_len -= 1

            tokenized_full_prompt["labels"] = [
                -100
            ] * user_prompt_len + tokenized_full_prompt["labels"][
                user_prompt_len:
            ]  # could be sped up, probably
        return tokenized_full_prompt


    # model = prepare_model_for_int8_training(model)
    model.resize_token_embeddings(len(tokenizer))
    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model.enable_input_require_grads()
    model = get_peft_model(model, config)

    # if data_path.endswith(".json") or data_path.endswith(".jsonl") or 'baseline' in data_path:
    train_data = CodeDataset(data_path+'apps_train.jsonl')
    val_data = CodeDataset(data_path + 'apps_test.jsonl')

    train_data.map(generate_and_tokenize_prompt)
    val_data.map(generate_and_tokenize_prompt)
    print("data 완료")
    # test_data.map(generate_and_tokenize_prompt)

    if resume_from_checkpoint:
        # Check the available weights and load them
        checkpoint_name = os.path.join(
            resume_from_checkpoint, "pytorch_model.bin"
        )  # Full checkpoint
        if not os.path.exists(checkpoint_name):
            checkpoint_name = os.path.join(
                resume_from_checkpoint, "adapter_model.bin"
            )  # only LoRA model - LoRA config above has to fit
            resume_from_checkpoint = (
                False  # So the trainer won't try loading its state
            )
        # The two files above have a different name depending on how they were saved, but are actually the same.
        if os.path.exists(checkpoint_name):
            print(f"Restarting from {checkpoint_name}")
            adapters_weights = torch.load(checkpoint_name)
            set_peft_model_state_dict(model, adapters_weights)
        else:
            print(f"Checkpoint {checkpoint_name} not found")

    model.print_trainable_parameters()  # Be more transparent about the % of trainable params.

    if not ddp and torch.cuda.device_count() > 1:
        # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
        model.is_parallelizable = True
        model.model_parallel = True

    trainer = transformers.Seq2SeqTrainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        # callbacks=[LogCallback],
        args=transformers.Seq2SeqTrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            per_device_eval_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=0,
            weight_decay=0.01, ## 새롭게 설정
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=1,
            logging_dir='./logs',
            optim="adamw_torch",
            lr_scheduler_type = 'cosine',
            evaluation_strategy="steps" if val_set_size > 0 else "no",
            save_strategy="steps",
            eval_steps=3 if val_set_size > 0 else None,
            save_steps=3,
            output_dir=output_dir,
            save_total_limit=3,
            load_best_model_at_end=True if val_set_size > 0 else False,
            ddp_find_unused_parameters=False if ddp else None,
            group_by_length=group_by_length,
            report_to="wandb" if use_wandb else None,
            run_name=wandb_run_name if use_wandb else None,
        ),
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=1, return_tensors="pt", padding=False
        )
    )


    model.config.use_cache = False

    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    trainer.train(resume_from_checkpoint=resume_from_checkpoint)

    model.save_pretrained(output_dir)
    model = model.merge_and_unload()

    # temp=[]
    # predictions = trainer.predict(test_data, max_length=1024)
    # preds = np.argmax(predictions.predictions, axis=-1)
    # for row in preds:
    #     temp.append({"pred":tokenizer.decode(row, skip_special_tokens=True)})
    # with jsonlines.open('./predict_check.jsonl', mode='w') as writer:
    #     for prefix in tqdm(temp):
    #         writer.write(prefix)

    # model.generate()

fire.Fire(train)



Training Alpaca-LoRA model with params:
base_model: microsoft/phi-1_5
data_path: ./
output_dir: /content/drive/MyDrive/out/first/
batch_size: 128
micro_batch_size: 1
num_epochs: 3
learning_rate: 0.0003
cutoff_len: 1024
val_set_size: 2000
lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['Wqkv', 'out_proj']
train_on_inputs: False
add_eos_token: True
group_by_length: False
wandb_project: dlproject
wandb_run_name: first
wandb_watch: true
wandb_log_model: true
resume_from_checkpoint: False
prompt template: no_instruction



wandb: Currently logged in as: xuio. Use `wandb login --relogin` to force relogin


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

RuntimeError: ignored

In [ ]:
# from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
# import torch.nn.init as init  # Importing the init module from PyTorch
# import bitsandbytes as bnb
# from contextlib import contextmanager

# def noop (x=None, *args, **kwargs):
#     "Do nothing"
#     return x

@contextmanager
# def no_kaiming():
#     old_iku = init.kaiming_uniform_
#     init.kaiming_uniform_ = noop
#     try: yield
#     finally: init.kaiming_uniform_ = old_iku

# _old_8init = Linear8bitLt.__init__
# _old_4init = Linear4bit.__init__

# def _new_4init(self, input_features, output_features, bias=True,
#                device=None, **kwargs):
#     with no_kaiming():
#         return _old_4init(self, input_features, output_features, bias=bias,
#                           device=device, **kwargs)



# def _new_8init(self, input_features, output_features, bias=True, has_fp16_weights=True,
#               memory_efficient_backward=False, threshold=0.0, index=None, device=None):
#     with no_kaiming():
#         return _old_8init(self, input_features, output_features, bias=bias, has_fp16_weights=has_fp16_weights,
#                           memory_efficient_backward=memory_efficient_backward, threshold=threshold, index=index, device=device)

# Linear8bitLt.__init__ = _new_8init
# Linear4bit.__init__ = _new_4init

In [ ]:
import os
import sys

import fire
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import jsonlines
from tqdm import tqdm
from datasets import load_dataset
# from utils.callbacks import Iteratorize, Stream
# from utils.prompter import Prompter

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:  # noqa: E722
    pass


def main(
    load_8bit: bool = False,
    base_model: str = "microsoft/phi-1_5",
    lora_weights: str = "/content/out/checkpoint-300/",
    prompt_template: str = "no_instruction",  # The prompt template to use, will default to alpaca.
):
    # base_model = base_model or os.environ.get("BASE_MODEL", "")
    assert (
        base_model
    ), "Please specify a --base_model, e.g. --base_model='huggyllama/llama-7b'"

    prompter = Prompter(prompt_template)
    # 이 부분 신경써야함
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = "left"  # Allow batched inference

    # if device == "cuda":

    # model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code=True, torch_dtype=torch.float32)

    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
    )
    model = PeftModel.from_pretrained(
        model,
        lora_weights,
        torch_dtype=torch.float16,
    )
    model.resize_token_embeddings(len(tokenizer))

    model.config.pad_token_id = tokenizer.pad_token_id  # unk
    model.config.bos_token_id = tokenizer.bos_token_id  # <s>
    model.config.eos_token_id = tokenizer.eos_token_id  # </s>


    if not load_8bit:
        model.half()  # seems to fix bugs for some users.

    model.eval()
    if torch.__version__ >= "2" and sys.platform != "win32":
        model = torch.compile(model)

    def evaluate(
        input,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=1,
        max_new_tokens=1024,
        **kwargs,
    ):
        # few-shot시 사용
        # prompt = "### Input:\n" + origin_data[1668]['description'] + "\n\n### Response:" + data[0]['label'] + "\n" + "### Input:\n" + origin_data[3381]['description'] + "\n\n### Response:" + data[1]['label'] + "\n" + input

        # SFT시 사용
        prompt = prompter.generate_prompt(input=input)
        inputs = tokenizer(prompt,
                            padding=False,
                            # truncation = True,
                            # max_length=1024,
                            return_tensors="pt",
                            )
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        generation_config = GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_beams=num_beams,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            **kwargs,
        )

        generate_params = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "generation_config": generation_config,
            "return_dict_in_generate": True,
            "output_scores": True,
            "max_new_tokens": max_new_tokens,
            "max_length" : max_new_tokens
        }

        with torch.no_grad():
            generation_output = model.generate(
                input_ids=generate_params['input_ids'],
                attention_mask=generate_params['attention_mask'],
                generation_config=generate_params['generation_config'],
                do_sample=True,
                return_dict_in_generate=True,
                output_scores=True,
                max_length=generate_params['max_length'],
                max_new_tokens=generate_params['max_new_tokens'],
            )
        output = tokenizer.decode(generation_output['sequences'][0], skip_special_tokens=True)
        return output

    data = []
    with jsonlines.open('./apps_train.jsonl') as reader:
        for pred_item in tqdm(reader):
            data.append(pred_item)
    for row in tqdm(data):
        instruction = row['description']
        result = evaluate(instruction)
        row['pred'] = result
    with jsonlines.open('./predict_check.jsonl', mode='w') as writer:
        for prefix in tqdm(data):
            writer.write(prefix)

if __name__ == "__main__":
    fire.Fire(main)

1it [00:00, 2164.24it/s]
100%|██████████| 1/1 [00:00<00:00, 1820.44it/s]
ERROR: Could not consume arg: -f
Usage: colab_kernel_launcher.py -

For detailed information on this command, run:
  colab_kernel_launcher.py - --help


FireExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
